In [1]:
import requests
import cnfg
import json
import pickle
import time
from datetime import datetime
from collections import defaultdict
from urlparse import urlparse
from pymongo import MongoClient

In [2]:
config = cnfg.load(".genius_config")
client_key = config["client_key"]
client_secret = config["client_secret"]
access_token = 'uK247ank6jF7DY_E7TEMIdbuhT2IBVsVeFXh6AluxswFUfrlFNfXNu2pUFPrGv2c'
auth = 'Bearer '+access_token
req_start = 'https://api.genius.com'

In [3]:
client = MongoClient()
db = client.music
s = db.songs
# counter
c = s.find()

In [4]:
def pickleLoad(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f) 
    return data

def pickleDump(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)

In [5]:
def api_call(validurl):
    call = validurl
    r = requests.get(call, headers={'Authorization':auth})
    if r.status_code==200:
        return json.loads(r.content)
    else:
        print r.content
        return r.content

In [6]:
def get_annotations(song):
    referenced = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/referents?song_id='+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for ref in refs_json['response']['referents']:
        for note in ref['annotations']:
            for child in note['body']['dom']['children']:
                if 'children' in child:
                    for element in child['children']:
                        if 'attributes' in element:
                            if 'href' in element['attributes']:
                                parsed = urlparse(element['attributes']['href'])
                                if parsed.netloc=="genius.com":
                                    try:
                                        apiurl = req_start+element['data']['api_path']
                                        annotype = urlparse(apiurl).path.split('/')[1]
                                        if annotype == 'albums':
                                            referenced['albums'].append(apiurl.encode('utf-8'))
                                        elif annotype == 'artists':
                                            referenced['artists'].append(apiurl.encode('utf-8'))
                                        elif annotype == 'songs':
                                            if urlparse(element['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                                referenced['songs'].append(apiurl.encode('utf-8'))
                                            else:
                                                referenced['other'].append(apiurl.encode('utf-8'))
                                    except:
                                        continue
                                            
    return dict(referenced)

In [7]:
def description_refs(song):
    description = defaultdict(list)
    if type(song)==int:
        myrequest = req_start+'/songs/'+str(song)
        r = requests.get(myrequest, headers={'Authorization':auth})
        refs_json = json.loads(r.content)
    elif type(song)==dict:
        refs_json = song
    for paragraph in refs_json['response']['song']['description']['dom']['children']:
        if 'children' in paragraph:
            for child in paragraph['children']:
                if 'attributes' in child:
                    if 'href' in child['attributes']:
                        url = child['attributes']['href']
                        if urlparse(url).netloc=='genius.com':
                            try:
                                apiurl = req_start+child['data']['api_path']
                                annotype = urlparse(apiurl).path.split('/')[1]
                                if annotype == 'albums':
                                    description['albums'].append(apiurl.encode('utf-8'))
                                elif annotype == 'artists':
                                    description['artists'].append(apiurl.encode('utf-8'))
                                elif annotype == 'songs':
                                    if urlparse(child['attributes']['href']).path.split('/')[-1][-6:] == 'lyrics':
                                        description['songs'].append(apiurl.encode('utf-8'))
                                    else:
                                        description['other'].append(apiurl.encode('utf-8'))
                            except:
                                continue
    return dict(description)

In [8]:
# pass one of the two following:
# song['references'] for referents or
# song['description_refs'] for references in description

def get_neighbors(song):
    neighbors = defaultdict(list)
    for item, vals in song.iteritems():
        if item=='artists':
            for artist in vals:
                artistid = artist.rpartition('/')[2]
                if s.find({'featured_artists': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'featured_artists': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'featured_artists': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['featured_artists'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'writers': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'writers': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'writers': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['writers'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'producers': {'$elemMatch':{'id':int(artistid)}}}).count():
                    name = s.find({'producers': {'$elemMatch':{'id':int(artistid)}}}, 
                                  {'producers': {'$elemMatch':{'id':int(artistid)}}}).next()
                    artistname = name['producers'][0]['artist']
                    neighbors['artist'].append({'name': artistname, 'id': artistid})
                elif s.find({'artist_id': int(artistid)}, {'artist':1}).count():
                    name = s.find({'artist_id': int(artistid)}, {'artist': 1}).next()
                    neighbors['artist'].append({'name': name['artist'], 'id': artistid})
                else:
#                     print "calling artistid: ", artistid
                    time.sleep(7)
                    name = api_call(artist)['response']['artist']['name']
#                     print "successful request!"
#                     print "artist name: ", name
                    neighbors['artist'].append({'name': name, 'id': artistid})
        elif item == 'albums':
            for album in vals:
                albumid = album.rpartition('/')[2]
                if s.find({'album.id': int(albumid)}).count():
                    name = s.find({'album.id': int(albumid)}, {'album.name': 1}).next()
                    albumname = name['album']['name']
                    neighbors['album'].append({'name': albumname, 'id': albumid})
                else:
#                     print "calling albumid: ", albumid
                    time.sleep(7)
                    name = api_call(album)['response']['album']['name']
#                     print "successful request!"
#                     print "album name: ", name
                    neighbors['album'].append({'name': name, 'id': albumid})
        elif item == 'songs':
            for song in vals:
                songid = song.rpartition('/')[2]
                if s.find({'id': int(songid)}).count(): 
                    data = s.find({'id': int(songid)}, {'title':1, 'artist':1}).next()
                    title = data['title']
                    artist = data['artist']
                    neighbors['song'].append({'title':title, 'artist':artist, 'id':songid})
                else:
                    neighbors['song_unannotated'].append({'id':songid})
        elif item == 'other':
            for other in vals:
                otherid = other.rpartition('/')[2]
#                 print "calling other: ", other
                name = api_call(other)['response']['song']['title']
                neighbors['other'].append({'title':name, 'id':otherid})
                
    return neighbors

In [9]:
def annotate_song(songid):
    if s.find({'id':songid}).count() > 0:
#         print "already in Mongo... returning Mongo query"
        return s.find({'id':songid}).next()
    else:
        song_info = {}
        myrequest = req_start+'/songs/'+str(songid)
        r = requests.get(myrequest, headers={'Authorization':auth})
        song_json = json.loads(r.content)
        song_info['title'] = str(song_json['response']['song']['title'])
        song_info['artist'] = str(song_json['response']['song']['primary_artist']['name'])
        song_info['artist_id'] = song_json['response']['song']['primary_artist']['id']
        pickle_title = '_'.join(song_info['title'].split())+"_"+'_'.join(song_info['artist'].split())+".pkl"
        pickleDump(song_json, pickle_title)
        if song_json['response']['song']['album'] != None:
            song_info['album'] = {'name': str(song_json['response']['song']['album']['name']), 
                              'id': song_json['response']['song']['album']['id']}
        song_info['id'] = song_json['response']['song']['id']

        # may or may not exist
        song_info['featured_artists'] = []
        song_info['writers'] = []
        song_info['producers'] = []
        
        if song_json['response']['song']['sampled_songs']:
            song_info['sampled_songs'] = song_json['response']['song']['sampled_songs']
        if song_json['response']['song']['sampling_songs']:
            song_info['sampling_songs'] = song_json['response']['song']['sampling_songs']

        for artist in song_json['response']['song']['featured_artists']:
            if artist['name']!=song_info['artist']:
                song_info['featured_artists'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['producer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['producers'].append({'artist':artist['name'], 'id':artist['id']})
        for artist in song_json['response']['song']['writer_artists']:
            if artist['name']!=song_info['artist']:
                song_info['writers'].append({'artist':artist['name'], 'id':artist['id']})

        if not song_info['featured_artists']:
            del song_info['featured_artists']
        if not song_info['writers']:
            del song_info['writers']
        if not song_info['producers']:
            del song_info['producers']

        # references!!
        time.sleep(7)
        song_info['references'] = dict(get_neighbors(get_annotations(songid)))
        time.sleep(7)
        song_info['description_refs'] = dict(get_neighbors(description_refs(song_json)))
        
        pickled_annotation = pickle_title[0:-4]+"_annotated.pkl"
        pickleDump(song_info, pickled_annotation)
        return song_info

In [10]:
newsongs = [1531904,
 550920,
 620555,
 2396178,
 2284206,
 1839127,
 51227,
 526365,
 508933,
 2218017,
 2082,
 2310181,
 245798,
 520237,
 2416699,
 1953852,
 61,
 38978,
 730057,
 68,
 2283589,
 414018,
 1966157,
 1378382,
 92856,
 4186,
 2214586,
 2263139,
 69732,
 51301,
 835688,
 108,
 206963,
 215157,
 690295,
 2171,
 118483,
 1200259,
 897159,
 436363,
 1317005,
 446608,
 1824913,
 2196,
 1978519,
 2204,
 2437280,
 88225,
 65701,
 2275494,
 168,
 4270,
 1742877,
 102576,
 181,
 1128630,
 2368201,
 51388,
 2339009,
 192545,
 235725,
 911574,
 2001112,
 30941,
 2177246,
 32995,
 233,
 2286,
 2308338,
 156373,
 592901,
 72405,
 20525,
 1478928,
 405779,
 2414874,
 2255131,
 4381,
 1548580,
 1394985,
 440618,
 33067,
 379183,
 2423091,
 2138429,
 4417,
 1820994,
 28995,
 2435401,
 2390346,
 829090,
 737615,
 518484,
 387414,
 176475,
 2277244,
 958524,
 672108,
 192879,
 375,
 70012,
 1935742,
 2275721,
 29066,
 2443,
 396,
 622989,
 414099,
 47508,
 65947,
 35229,
 2275742,
 29087,
 41376,
 2177091,
 672155,
 1778090,
 399792,
 1778100,
 436662,
 444,
 2431426,
 688579,
 452,
 170443,
 2374092,
 82339,
 35293,
 680415,
 57824,
 733666,
 35301,
 1847784,
 2402796,
 207343,
 725500,
 4609,
 518,
 713548,
 78347,
 526,
 395791,
 2284049,
 16914,
 4636,
 2433565,
 1114654,
 2435615,
 4640,
 2400811,
 2609,
 2620,
 2433598,
 2302528,
 45634,
 2302535,
 45643,
 2430733,
 35414,
 2042461,
 210364,
 1790573,
 2275954,
 709240,
 634,
 515519,
 2112517,
 2421373,
 2329215,
 131712,
 731783,
 696428,
 51652,
 2210470,
 393899,
 131758,
 690,
 2124472,
 416444,
 92862,
 2142915,
 2396871,
 2423497,
 2409162,
 32546,
 718,
 694992,
 2057145,
 101077,
 58072,
 2388703,
 2155243,
 115451,
 725757,
 361214,
 2433799,
 338701,
 506641,
 1415961,
 66330,
 121627,
 117534,
 1891103,
 33573,
 1995564,
 33581,
 2423601,
 912179,
 822,
 826,
 1696577,
 174916,
 2892,
 446605,
 27472,
 160600,
 29529,
 47967,
 185191,
 47982,
 437103,
 880,
 2327412,
 2171992,
 1989637,
 246656,
 123873,
 2325382,
 2028423,
 279437,
 1850256,
 1702804,
 158615,
 1465241,
 2275787,
 66467,
 2397096,
 1392455,
 82860,
 1172403,
 27574,
 1782713,
 76731,
 2328394,
 209861,
 2413513,
 433098,
 293841,
 667130,
 2325471,
 848,
 2311139,
 1182694,
 115688,
 58346,
 2163879,
 50156,
 431093,
 49680,
 60408,
 2270213,
 610479,
 6320,
 1060,
 2081829,
 637994,
 349229,
 1629231,
 502834,
 230455,
 578626,
 1420355,
 1971271,
 302157,
 547918,
 2296914,
 1674324,
 1555542,
 494680,
 1113,
 668175,
 2292829,
 746593,
 390244,
 720401,
 2417511,
 466028,
 709741,
 279669,
 15479,
 312444,
 2413699,
 93317,
 1940620,
 56477,
 1954979,
 3239,
 2329769,
 11435,
 459948,
 1799343,
 23070,
 640182,
 1973431,
 361656,
 31934,
 2311363,
 2311373,
 373966,
 1240,
 2307291,
 1559777,
 2286821,
 38118,
 468201,
 13556,
 1942775,
 1277,
 2426112,
 380162,
 19715,
 410884,
 722193,
 2295060,
 44311,
 376091,
 1860893,
 707870,
 2153759,
 658724,
 1318,
 30258,
 521525,
 2297738,
 392510,
 357695,
 48453,
 2315590,
 699959,
 94434,
 13648,
 378195,
 13656,
 2409818,
 1779037,
 1944935,
 533873,
 389694,
 451958,
 722296,
 19348,
 38270,
 720259,
 433542,
 62862,
 2332048,
 23955,
 583,
 310705,
 290230,
 110010,
 2119099,
 2262460,
 329151,
 38340,
 476615,
 56776,
 112079,
 417235,
 1791444,
 1493,
 591,
 2086372,
 114153,
 1517,
 703741,
 1620904,
 372211,
 2018804,
 556538,
 9723,
 245249,
 403031,
 2262544,
 914961,
 1283,
 2117152,
 513574,
 7722,
 3627,
 226866,
 130616,
 1637950,
 56902,
 2055751,
 1931490,
 1612,
 52814,
 8120,
 156955,
 742999,
 720473,
 1869407,
 1980001,
 394854,
 2295400,
 1574514,
 2145913,
 118395,
 210558,
 491142,
 89736,
 2434697,
 693900,
 2024045,
 2416274,
 2100885,
 2176665,
 2123421,
 607906,
 1365667,
 1242795,
 122542,
 54967,
 523961,
 2408122,
 2176703,
 124611,
 2178762,
 5835,
 700112,
 20179,
 218837,
 601817,
 607964,
 743136,
 1666786,
 124643,
 2373349,
 428668,
 2178795,
 2275965,
 319225,
 114429,
 2228864,
 1560323,
 452357,
 716553,
 1805,
 1820438,
 983001,
 2342690,
 2332455,
 2420521,
 1541932,
 2324269,
 698675,
 2268248,
 51003,
 734517,
 1943366,
 2414561,
 118602,
 87884,
 2287448,
 2414425,
 2408283,
 200546,
 61287,
 282475,
 409461,
 24440,
 741241,
 186250,
 729410,
 3984,
 2254750,
 307105,
 427939,
 206760,
 2068393,
 1996715,
 2334638,
 530356,
 2308425,
 51128,
 1800122,
 92490,
 2181057,
 2379715,
 33782,
 1993,
 1698,
 73037,
 1935314,
 2205656,
 2424793,
 1978332,
 71649,
 1986530,
 57315,
 1402857,
 2215917]

In [11]:
unable_to_insert = []

In [12]:
for songid in newsongs:
    if s.find({'id':songid}).count()>0:
#         print "song id ", songid, " in Mongo already"
        continue
    else:
        try:
            annotated = annotate_song(songid)
            s.insert_one(annotated)
            print "!!!inserted ", songid, " to Mongo!!!!"
            time.sleep(7)
        except:
            unable_to_insert.append(songid)
            continue

!!!inserted  680415  to Mongo!!!!
!!!inserted  57824  to Mongo!!!!
!!!inserted  733666  to Mongo!!!!
!!!inserted  35301  to Mongo!!!!
!!!inserted  1847784  to Mongo!!!!
!!!inserted  2402796  to Mongo!!!!
!!!inserted  207343  to Mongo!!!!
!!!inserted  725500  to Mongo!!!!
!!!inserted  4609  to Mongo!!!!
!!!inserted  518  to Mongo!!!!
!!!inserted  78347  to Mongo!!!!
!!!inserted  526  to Mongo!!!!
!!!inserted  395791  to Mongo!!!!
!!!inserted  2284049  to Mongo!!!!
!!!inserted  16914  to Mongo!!!!
!!!inserted  4636  to Mongo!!!!
!!!inserted  2433565  to Mongo!!!!
!!!inserted  1114654  to Mongo!!!!
!!!inserted  2435615  to Mongo!!!!
!!!inserted  4640  to Mongo!!!!
!!!inserted  2400811  to Mongo!!!!
!!!inserted  2609  to Mongo!!!!
!!!inserted  2620  to Mongo!!!!
!!!inserted  2433598  to Mongo!!!!
!!!inserted  2302528  to Mongo!!!!
!!!inserted  45634  to Mongo!!!!
!!!inserted  2302535  to Mongo!!!!
!!!inserted  45643  to Mongo!!!!
!!!inserted  2430733  to Mongo!!!!
!!!inserted  35414  to Mo

In [27]:
to_be_updated = [('Summer_Sixteen_Drake.pkl', 'Summer_Sixteen_Drake_annotated.pkl'), 
('Hercules_Young_Thug.pkl', 'Hercules_Young_Thug_annotated.pkl'),
('Solo_ILoveMakonnen.pkl', 'Solo_ILoveMakonnen_annotated.pkl'),
('Bake_Sale_Wiz_Khalifa.pkl', 'Bake_Sale_Wiz_Khalifa_annotated.pkl'),
('Low_Life_Future.pkl', 'Low_Life_Future_annotated.pkl'),
('Love_Yourz_J._Cole.pkl', 'Love_Yourz_J._Cole_annotated.pkl'),
('Work_Rihanna.pkl', 'Work_Rihanna_annotated.pkl'),
('Gotta_Lotta_2_Chainz_&_Lil_Wayne.pkl', 'Gotta_Lotta_2_Chainz_&_Lil_Wayne_annotated.pkl'),
('Saved_Ty_Dolla_$ign.pkl', 'Saved_Ty_Dolla_$ign_annotated.pkl'),
('New_Level_A$AP_Ferg.pkl', 'New_Level_A$AP_Ferg_annotated.pkl'),
('2_Phones_Kevin_Gates.pkl', '2_Phones_Kevin_Gates_annotated.pkl'),
('Cut_It_O.T._Genasis.pkl', 'Cut_It_O.T._Genasis_annotated.pkl'),
('Plug_Rich_The_Kid.pkl', 'Plug_Rich_The_Kid_annotated.pkl'),
('Law_Yo_Gotti.pkl', 'Law_Yo_Gotti_annotated.pkl'),
('Best_Friend_Young_Thug.pkl', 'Best_Friend_Young_Thug_annotated.pkl'),
('Pass_Dat_Jeremih.pkl', 'Pass_Dat_Jeremih_annotated.pkl'),
('Exchange_Bryson_Tiller.pkl', 'Exchange_Bryson_Tiller_annotated.pkl'),
('Ready_Chaz_French.pkl', 'Ready_Chaz_French_annotated.pkl'),
('Weekend_Mac_Miller.pkl', 'Weekend_Mac_Miller_annotated.pkl'),
('Ladies_Hit_Squad_Skepta.pkl', 'Ladies_Hit_Squad_Skepta_annotated.pkl'),
('Moolah_Young_Greatness.pkl', 'Moolah_Young_Greatness_annotated.pkl'),
('Walked_In_Bankroll_Fresh.pkl', 'Walked_In_Bankroll_Fresh_annotated.pkl'),
('Sorry_Not_Sorry_Bryson_Tiller.pkl', 'Sorry_Not_Sorry_Bryson_Tiller_annotated.pkl'),
('Photo_Copied_Future.pkl', 'Photo_Copied_Future_annotated.pkl'),
('By_Chance_Mike_WiLL_Made_It.pkl', 'By_Chance_Mike_WiLL_Made_It_annotated.pkl'),
('Key_To_The_Streets_YFN_Lucci.pkl', 'Key_To_The_Streets_YFN_Lucci_annotated.pkl'),
('Money_Talk_T.I..pkl', 'Money_Talk_T.I._annotated.pkl'),
('Top_Lil_Uzi_Vert.pkl', 'Top_Lil_Uzi_Vert_annotated.pkl'),
("Pushin'_ILoveMakonnen.pkl", "Pushin'_ILoveMakonnen_annotated.pkl"),
('Bounce_2_Chainz_&_Lil_Wayne.pkl', 'Bounce_2_Chainz_&_Lil_Wayne_annotated.pkl'),
('Slugs_Trae_tha_Truth.pkl', 'Slugs_Trae_tha_Truth_annotated.pkl')]

In [36]:
def update_annotation(song):
    song_json = pickleLoad(song[0])
    song_info = pickleLoad(song[1])
    
    if song_json['response']['song']['sampled_songs']:
        song_info['sampled_songs'] = song_json['response']['song']['sampled_songs']
    if song_json['response']['song']['sampling_songs']:
        song_info['sampling_songs'] = song_json['response']['song']['sampling_songs']
        
    song_info['artist'] = str(song_json['response']['song']['primary_artist']['name'])
    song_info['artist_id'] = song_json['response']['song']['primary_artist']['id']
    
    return song_info

In [37]:
summersixteen = update_annotation(to_be_updated[0])

In [42]:
summersixteen['id']

2415289

In [40]:
s.update_one({'id':2415289}, summersixteen)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  if __name__ == '__main__':


{u'n': 1, u'nModified': 1, u'ok': 1, 'updatedExisting': True}

In [41]:
s.find({'id':2415289}).next()

{u'_id': ObjectId('56ef4e178957421b672a9613'),
 u'album': {u'id': 132506, u'name': u'Views From The 6'},
 u'artist': u'Drake',
 u'artist_id': 130,
 u'description_refs': {u'album': [{u'id': u'132506',
    u'name': u'Views From The 6'}],
  u'other': [{u'id': u'2415314', u'title': u'War Pain'}],
  u'song_unannotated': [{u'id': u'2413886'}]},
 u'id': 2415289,
 u'producers': [{u'artist': u'Noah \u201c40\u201d Shebib', u'id': 27612},
  {u'artist': u'Boi-1da', u'id': 662},
  {u'artist': u'Brian Bennett', u'id': 634949},
  {u'artist': u'Cubeatz', u'id': 32429}],
 u'references': {u'album': [{u'id': u'120604', u'name': u'The Life of Pablo'}],
  u'artist': [{u'id': u'1632', u'name': u'Tory Lanez'},
   {u'id': u'1632', u'name': u'Tory Lanez'},
   {u'id': u'27638', u'name': u'KeY Wane'},
   {u'id': u'1319', u'name': u'Meek Mill'}],
  u'other': [{u'id': u'1791', u'title': u'Gorgeous'}],
  u'song_unannotated': [{u'id': u'2310122'},
   {u'id': u'140752'},
   {u'id': u'217266'},
   {u'id': u'2396836'},

In [43]:
s.count()

43

In [44]:
for thesong in to_be_updated:
    try:
        newanno = update_annotation(thesong)
        songid = newanno['id']
        s.update_one({'id':songid}, newanno)
    except:
        "unable to update song: ", thesong[0]

In [45]:
s.count()

43

In [46]:
s.find({'description_refs.song_unnannotated': {'$exists': true}},
        {'description_refs.song_unannotated'})

{u'_id': ObjectId('56ef3685895742130ba296f8'),
 u'album': {u'id': 129747, u'name': u'Fetty Wap '},
 u'artist': u'Fetty Wap',
 u'artist_id': 216609.0,
 u'description_refs': {u'song_unannotated': [{u'id': u'496445'}]},
 u'featured_artists': [{u'artist': u'Monty', u'id': 19635}],
 u'id': 653082,
 u'producers': [{u'artist': u'JayFrance', u'id': 557945},
  {u'artist': u'Nick EBeats', u'id': 72396}],
 u'references': {u'artist': [{u'id': u'7031', u'name': u'Luther Vandross'}],
  u'song_unannotated': [{u'id': u'496445'}]},
 u'title': u'My Way',
 u'writers': [{u'artist': u'Remy Boyz', u'id': 211094}]}

In [95]:
s.find({'description_refs.song_unannotated': {'$eq':653082}}).next()

{u'_id': ObjectId('56ef3685895742130ba296f8'),
 u'album': {u'id': 129747, u'name': u'Fetty Wap '},
 u'artist': u'Fetty Wap',
 u'artist_id': 216609.0,
 u'description_refs': {u'song_unannotated': [{u'id': u'496445'}]},
 u'featured_artists': [{u'artist': u'Monty', u'id': 19635}],
 u'id': 653082,
 u'producers': [{u'artist': u'JayFrance', u'id': 557945},
  {u'artist': u'Nick EBeats', u'id': 72396}],
 u'references': {u'artist': [{u'id': u'7031', u'name': u'Luther Vandross'}],
  u'song_unannotated': [{u'id': u'496445'}]},
 u'title': u'My Way',
 u'writers': [{u'artist': u'Remy Boyz', u'id': 211094}]}

In [96]:
s.find({'description_refs.song_unannotated': {'$elemMatch':{'id':'496445'}}}).next()

{u'_id': ObjectId('56ef3685895742130ba296f8'),
 u'album': {u'id': 129747, u'name': u'Fetty Wap '},
 u'artist': u'Fetty Wap',
 u'artist_id': 216609.0,
 u'description_refs': {u'song_unannotated': [{u'id': u'496445'}]},
 u'featured_artists': [{u'artist': u'Monty', u'id': 19635}],
 u'id': 653082,
 u'producers': [{u'artist': u'JayFrance', u'id': 557945},
  {u'artist': u'Nick EBeats', u'id': 72396}],
 u'references': {u'artist': [{u'id': u'7031', u'name': u'Luther Vandross'}],
  u'song_unannotated': [{u'id': u'496445'}]},
 u'title': u'My Way',
 u'writers': [{u'artist': u'Remy Boyz', u'id': 211094}]}

In [76]:
more_songs = []

In [407]:
c = s.find({'description_refs.song_unannotated': {'$exists': 'true'}},
        {'description_refs.song_unannotated': 1, '_id':0})

In [406]:
r = s.find({'references.song_unannotated': {'$exists': 'true'}},
        {'references.song_unannotated': 1, '_id':0})

In [400]:
supposedly_unannotated = []

In [408]:
for i in list(c):
    for song in i['description_refs']['song_unannotated']:
        supposedly_unannotated.append(int(song['id']))
for i in list(r):
    for song in i['references']['song_unannotated']:
        supposedly_unannotated.append(int(song['id']))

In [81]:
for i in list(c):
    for song in i['description_refs']['song_unannotated']:
        more_songs.append(int(song['id']))
for i in list(r):
    for song in i['references']['song_unannotated']:
        more_songs.append(int(song['id']))

In [116]:
list(s.find({'description_refs.song_unannotated':{'$elemMatch':{'id':str(2413886)}}}))

[{u'_id': ObjectId('56ef4e178957421b672a9613'),
  u'album': {u'id': 132506, u'name': u'Views From The 6'},
  u'artist': u'Drake',
  u'artist_id': 130,
  u'description_refs': {u'album': [{u'id': u'132506',
     u'name': u'Views From The 6'}],
   u'other': [{u'id': u'2415314', u'title': u'War Pain'}],
   u'song_unannotated': [{u'id': u'2413886'}]},
  u'id': 2415289,
  u'producers': [{u'artist': u'Noah \u201c40\u201d Shebib', u'id': 27612},
   {u'artist': u'Boi-1da', u'id': 662},
   {u'artist': u'Brian Bennett', u'id': 634949},
   {u'artist': u'Cubeatz', u'id': 32429}],
  u'references': {u'album': [{u'id': u'120604',
     u'name': u'The Life of Pablo'}],
   u'artist': [{u'id': u'1632', u'name': u'Tory Lanez'},
    {u'id': u'1632', u'name': u'Tory Lanez'},
    {u'id': u'27638', u'name': u'KeY Wane'},
    {u'id': u'1319', u'name': u'Meek Mill'}],
   u'other': [{u'id': u'1791', u'title': u'Gorgeous'}],
   u'song_unannotated': [{u'id': u'2310122'},
    {u'id': u'140752'},
    {u'id': u'217266

In [115]:
list(s.find({'description_refs.song_unannotated':{'$elemMatch':{'id':str(myway_songid)}}}))

[]

In [103]:
s.update({'references.song_unannotated':{'$elemMatch':{'id':str(myway_songid)}}},
         {'$set': {'references.song': 
                   {'id':str(myway_songid), 'title':ref['title'], 'artist':ref['artist']}
                  },
          '$unset': {'references.song_unannotated': {'$elemMatch':{'id':str(myway_songid)}}}
         }
)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


{u'n': 1, u'nModified': 1, u'ok': 1, 'updatedExisting': True}

In [111]:
s.update({'id':2396122},
        {'$set': {'references.song': [{'id':str(myway_songid), 'title':ref['title'], 'artist':ref['artist']}]
                }
        })

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  from ipykernel import kernelapp as app


{u'n': 1, u'nModified': 1, u'ok': 1, 'updatedExisting': True}

In [112]:
s.find({'id':2396122}).next()

{u'_id': ObjectId('56ef38dd895742130ba296f9'),
 u'album': {u'id': 140468, u'name': u'Summer In The Winter'},
 u'artist': u'Kid Ink',
 u'artist_id': 1631.0,
 u'description_refs': {u'album': [{u'id': u'140468',
    u'name': u'Summer In The Winter'}]},
 u'featured_artists': [{u'artist': u'Fetty Wap', u'id': 216609}],
 u'id': 2396122,
 u'producers': [{u'artist': u'DJ Mustard', u'id': 29780}],
 u'references': {u'album': [{u'id': u'10137',
    u'name': u'The King of Crunk & BME Recordings Present Lil Scrappy & Trillville'}],
  u'artist': [{u'id': u'17560', u'name': u'Ca$h Out'},
   {u'id': u'29780', u'name': u'DJ Mustard'},
   {u'id': u'134', u'name': u'Trillville'}],
  u'other': [{u'id': u'583442', u'title': u'Kill Bill- Opening Scene'}],
  u'song': [{u'artist': u'Fetty Wap', u'id': u'653082', u'title': u'My Way'}]},
 u'title': u'Promise'}

In [121]:
def insert_neighbors(neighbors):
    cant_insert = []
    for songid in neighbors:
        if s.find({'id':songid}).count()>0:
            print "song id ", songid, " in Mongo already"
            continue
        else:
            try:
                annotated = annotate_song(songid)
                s.insert_one(annotated)
                print "inserted ", songid, " to Mongo!!"
                to_insert = {'id':songid, 'title':annotated['title'], 'artist':annotated['artist']}
                s.update({'references.song_unannotated':{'$elemMatch':{'id':str(songid)}}},
                         {'$push': {'references.song': to_insert},
                          '$unset': {'references.song_unannotated': {'$elemMatch':{'id':str(songid)}}}
                         })
                s.update({'description_refs.song_unannotated':{'$elemMatch':{'id':str(songid)}}},
                 {'$push': {'description_refs.song': to_insert
                            },
                  '$unset': {'description_refs.song_unannotated': {'$elemMatch':{'id':str(songid)}}}
                 })
                time.sleep(7)
            except:
                cant_insert.append(songid)
                continue

In [118]:
insert_neighbors(more_songs)

calling artistid:  211094
artist name:  Remy Boyz
calling artistid:  216609
artist name:  Fetty Wap
calling artistid:  19635
artist name:  Monty
calling artistid:  324091
artist name:  P-Dice
inserted 

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:20: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


 496445  to Mongo!!
calling albumid:  132506
album name:  Views From The 6
inserted  2413886  to Mongo!!
calling other:  https://api.genius.com/songs/672
inserted  1812  to Mongo!!
inserted  58344  to Mongo!!
inserted  525473  to Mongo!!
calling artistid:  1630
artist name:  Ciara
inserted  2213836  to Mongo!!
inserted  583542  to Mongo!!
calling artistid:  338
artist name:  Busta Rhymes
calling artistid:  338
artist name:  Busta Rhymes
inserted  539654  to Mongo!!
song id  539654  in Mongo already
inserted  2274243  to Mongo!!
calling albumid:  143267
album name:  EVOL
calling albumid:  120604
album name:  The Life of Pablo
calling artistid:  626678
artist name:  Desiigner
calling artistid:  19139
artist name:  Shy Glizzy
inserted  2422513  to Mongo!!
calling albumid:  129455
album name:  T R A P S O U L
inserted  2315263  to Mongo!!
calling artistid:  2197
artist name:  Future
inserted  2172126  to Mongo!!
calling artistid:  59
artist name:  2Pac
inserted  2311370  to Mongo!!
song id

In [120]:
song

{u'id': u'2426159'}

In [122]:
cant_insert

NameError: name 'cant_insert' is not defined

In [123]:
len(more_songs) == len(set(more_songs))

False

In [124]:
more_inserted_songs = [496445, 2413886, 1812, 58344, 525473, 2213836, \
    583542, 539654, 539654, 2274243, 2422513, 2315263, 2172126, 2311370, \
    2213836, 2311366, 2399227, 558257, 2368510, 496445, 66225, 653082, \
    2310122, 217266, 2396836, 73301, 196551, 51360, 703742, 386316, \
    42966, 729458, 729458, 2311369, 2373, 599433, 648314, 240199, \
    2322011, 1932, 55599, 15961, 415574, 28749, 149774, 108547, 55662, \
    136893, 26798, 579968, 2440697, 4849, 454, 66, 497, 2315263, 579968, \
    579968, 507478, 423398, 602900, 2273460, 2342995, 101184, 569389, \
    2381771, 67051, 728784, 559442, 2214983, 499870, 2399227, 360730, \
    76654, 21902, 59024, 2426159]

In [361]:
updated_refs = [2396122, 2415289, 2369079, 2396302, 599451, 2398213, 2330692, 2392485, 2425602, 2165110, 2375319, 2312297, 2394578, 2305085, 2307164, 709256, 2401700, 2344225, 2413367, 2342860, 2439141, 2347598, 2399225, 2432858, 2312325, 507478]

In [360]:
updated_desc = [2415289, 2398213, 2392485, 2348651, 2295548, 2165110, 2422646, 709256, 2417517, 2399225, 2312325, 2426159, 507478]

In [127]:
len(inserted)

33

In [363]:
refs_pickles = []
desc_pickles = []
for docid in updated_desc:
    data = s.find({'id':docid}, {'title':1, 'artist':1, '_id':0}).next()
    title = data['title']
    artist = data['artist']
    picklename = '_'.join(title.split())+"_"+'_'.join(artist.split())+"_annotated.pkl"
    desc_pickles.append(picklename)
for docid in updated_refs:
    data = s.find({'id':docid}, {'title':1, 'artist':1, '_id':0}).next()
    title = data['title']
    artist = data['artist']
    picklename = '_'.join(title.split())+"_"+'_'.join(artist.split())+"_annotated.pkl"
    refs_pickles.append(picklename)

In [382]:
def fix_ref_annotations(picklesong):
    try:
        song_info = pickleLoad(picklesong)
        ref_songs = []
        failed_songs = []
        for element in song_info['references']['song_unannotated']:
            for k, v in element.iteritems():
                if s.find({'id':int(v)}).count() > 0:
                    vals = s.find({'id':int(v)}, {'artist':1, 'title':1, '_id':0}).next()
                    to_insert = {'id':v, 'artist':vals['artist'], 'title':vals['title']}
                    ref_songs.append(to_insert)
                else:
                    failed_songs.append({'id': v})     
        songid = song_info['id']
        if ref_songs and failed_songs:
            s.update_one({'id':songid}, {'$unset': {'references.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'references.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'references.song': {'$each': ref_songs}}}) 
            s.update_one({'id':songid}, {'$addToSet': {'references.song_unannotated': {'$each': failed_songs}}})
            song_info['references']['song'] = ref_songs
            song_info['references']['song_unannotated'] = failed_songs
        elif ref_songs and not failed_songs:
            s.update_one({'id':songid}, {'$unset': {'references.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'references.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'references.song': {'$each': ref_songs}}})
            song_info['references']['song'] = ref_songs
            del song_info['references']['song_unannotated']
        elif failed_songs and not ref_songs:
            s.update_one({'id':songid}, {'$unset': {'references.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'references.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'references.song_unannotated': {'$each': failed_songs}}})
            del song_info['references']['song']
            song_info['references']['song_unannotated'] = failed_songs
        
        pickleDump(song_info, picklesong)
        return True
    except:
        print "unable to update: ", picklesong
        return False

In [354]:
def fix_desc_annotations(picklesong):
    try: 
        song_info = pickleLoad(picklesong)
        desc_songs = []
        failed_songs = []
        for element in song_info['description_refs']['song_unannotated']:
            for k, v in element.iteritems():
                if s.find({'id':int(v)}).count() > 0:
                    vals = s.find({'id':int(v)}, {'artist':1, 'title':1, '_id':0}).next()
                    to_insert = {'id':v, 'artist':vals['artist'], 'title':vals['title']}
                    desc_songs.append(to_insert)
                else:
                    failed_songs.append({'id': v})
        songid = song_info['id']
        if desc_songs and failed_songs:
            s.update_one({'id':songid}, {'$unset': {'description_refs.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'description_refs.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song': {'$each': desc_songs}}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song_unannotated': {'$each': failed_songs}}})
            song_info['description_refs']['song'] = desc_songs
            song_info['description_refs']['song_unannotated'] = failed_songs
        elif desc_songs and not failed_songs:
            s.update_one({'id':songid}, {'$unset': {'description_refs.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'description_refs.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song': {'$each': desc_songs}}})
            song_info['description_refs']['song'] = desc_songs
            del song_info['description_refs']['song_unannotated']
        elif failed_songs and not desc_songs:
            s.update_one({'id':songid}, {'$unset': {'description_refs.song': ""}})
            s.update_one({'id':songid}, {'$unset': {'description_refs.song_unannotated':""}})
            s.update_one({'id':songid}, {'$addToSet': {'description_refs.song_unannotated': {'$each': failed_songs}}})
            del song_info['description_refs']['song']
            song_info['description_refs']['song_unannotated'] = failed_songs
        
        pickleDump(song_info, picklesong)
        return True
    except:
        print "unable to update: ", picklesong
        return False

In [374]:
s.find({'id':2396122}, {'references':1, 'title':1, 'artist':1}).next()

{u'_id': ObjectId('56ef38dd895742130ba296f9'),
 u'artist': u'Kid Ink',
 u'references': {u'album': [{u'id': u'10137',
    u'name': u'The King of Crunk & BME Recordings Present Lil Scrappy & Trillville'}],
  u'artist': [{u'id': u'17560', u'name': u'Ca$h Out'},
   {u'id': u'29780', u'name': u'DJ Mustard'},
   {u'id': u'134', u'name': u'Trillville'}],
  u'other': [{u'id': u'583442', u'title': u'Kill Bill- Opening Scene'}],
  u'song': [{u'artist': u'Ca$h Out', u'id': u'66225', u'title': u'Cashin Out'},
   {u'artist': u'Fetty Wap', u'id': u'653082', u'title': u'My Way'}]},
 u'title': u'Promise'}

In [367]:
refs_pickles[0]

'promise_kid_ink_info.pkl'

In [377]:
promise_info = pickleLoad(refs_pickles[0])

In [378]:
promise_info['references']

{'album': [{'id': '10137',
   'name': u'The King of Crunk & BME Recordings Present Lil Scrappy & Trillville'}],
 'artist': [{'id': '17560', 'name': u'Ca$h Out'},
  {'id': '29780', 'name': u'DJ Mustard'},
  {'id': '134', 'name': u'Trillville'}],
 'other': [{'id': '583442', 'title': u'Kill Bill- Opening Scene'}],
 'song': [{'artist': u'Ca$h Out', 'id': '66225', 'title': u'Cashin Out'},
  {'artist': u'Fetty Wap', 'id': '653082', 'title': u'My Way'}]}

In [376]:
fix_ref_annotations(refs_pickles[0])

unable to update:  promise_kid_ink_info.pkl


False

In [379]:
refs_pickles = refs_pickles[1:]

In [383]:
for p in refs_pickles:
    print fix_ref_annotations(p)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [384]:
for p in desc_pickles:
    print fix_desc_annotations(p)

True
True
True
True
True
True
True
True
True
True
True
True
True


In [409]:
supposedly_unannotated

[2339761,
 2396309,
 2417474,
 599395,
 2396309,
 21929,
 2415289,
 1812,
 437376,
 2400811,
 2291102,
 2399225,
 76103,
 73198,
 68615,
 73850,
 50025,
 729457,
 2273462,
 1913178,
 501510,
 639914,
 2322001,
 2322003,
 2322014,
 2322004,
 2322010,
 2322009,
 2322020,
 106362,
 949,
 423398,
 507478,
 2159541,
 2329765,
 2293108,
 2390938,
 46329,
 32795,
 42265,
 140752,
 156640,
 2266961,
 499835,
 460183,
 164344,
 2413931,
 151327,
 693724,
 2711,
 703732,
 2426159,
 168255,
 1800164,
 710897,
 140800,
 118141,
 2149919,
 57183,
 69579,
 555359,
 622818,
 517,
 2045,
 559003,
 2213109,
 2263723,
 494038,
 2290819,
 217263,
 217270,
 140752,
 1655,
 203961,
 50025,
 156640,
 703654,
 386121,
 42956,
 78169,
 542389,
 727466,
 542390,
 58349,
 118064,
 75115,
 2172126,
 2024226,
 703731,
 486501,
 69295,
 505193,
 2172126,
 1741,
 478469,
 732419,
 64,
 1016,
 58340,
 89740,
 2644,
 58497,
 79516,
 82701,
 17807,
 2149919,
 47702,
 704,
 68291,
 40840,
 3025,
 2695,
 698,
 569516,
 

In [411]:
len(set(supposedly_unannotated))

128